# REPORT: Predicting Risk of Heart Disease from Accessible Health Metrics

## Introduction:

According to the Public Health Agency of Canada, heart disease is the second leading cause of death in Canada, with approximately 1 in 12 Canadian adults over 20 living with a diagnosis. These metrics highlight the importance of knowing the risk factors and having access to medical advice. However, a shortage of physicians in Canada is causing a lack of available health care (Flood et al., 2023). Non-healthcare professionals do not have the means to properly self-evaluate symptoms, therefore our project seeks to help the general population to make informative decisions about heart disease symptoms that are self-monitored or easily accessible.


Thus we ask, is it possible to classify individuals into levels of heart disease risk (low risk, moderate risk, or high risk) based on blood pressure, cholesterol, heart rate and chest pain?


Our analysis will use the Heart Disease dataset from the Cleveland database for heart disease (Andras et al., 1988). This database consists of 303 patients without history of heart disease, who were admitted to the Cleveland Clinic between 1981 and 1984. 



In [1]:
# Please uncomment the following cell to install the altair in case your package is not up-to-date

In [2]:
# pip install -U altair

In [3]:
import altair as alt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import set_config
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler 
from sklearn.compose import make_column_transformer
from sklearn.utils import resample
from sklearn.pipeline import make_pipeline

In [4]:
# import dataset
heart_disease = pd.read_csv("https://archive.ics.uci.edu/static/public/45/data.csv")

# filter dataframe
heart_disease.rename(columns = {
                          "fbs" : "fasting_blood_sugar",
                          "chol" : "cholesterol", 
                          "cp":"type_chestpain",
                          "restecg" : "resting_ecg",
                          "thalach" : "max_heart_rate",
                          "exang" : "exercise_induced_angina",
                          "oldpeak" : "ST_depression", 
                          "slope" : "ST_segment_slope", 
                          "ca" : "num_major_vessels", 
                          "thal" : "thallium_stress_test", #not sure
                          "num" : "diagnosis",
                          "trestbps" : "resting_bp"
}, inplace = True)

heart_disease = heart_disease[["cholesterol","type_chestpain","max_heart_rate","resting_bp","diagnosis"]]


# A low-risk diagnosis is 0, 1
# A moderate-risk diagnosis is 2, 3
# A high-risk diagnosis is 4
heart_disease['diagnosis'] = heart_disease['diagnosis'].replace([0,1], "low-risk heart disease")
heart_disease['diagnosis'] = heart_disease['diagnosis'].replace([2,3], "moderate-risk heart disease")
heart_disease['diagnosis'] = heart_disease['diagnosis'].replace([4], "high-risk heart disease")

# chest pain type
heart_disease['type_chestpain'] = heart_disease['type_chestpain'].replace(
    [1,2,3,4],
    ["type1","type2","type3","type4"])

heart_disease['diagnosis'].value_counts()

low-risk heart disease         219
moderate-risk heart disease     71
high-risk heart disease         13
Name: diagnosis, dtype: int64

In [5]:
# balancing heart disease since we need more low risk and moderate risk diagnosis observations
rare_diagnosis_1 = heart_disease[heart_disease["diagnosis"] == "high-risk heart disease"]
rare_diagnosis_2 = heart_disease[heart_disease["diagnosis"] == "moderate-risk heart disease"]
low_risk_diagnosis = heart_disease[heart_disease["diagnosis"] == "low-risk heart disease"]

rare_diagnosis_upsample_1 = resample(rare_diagnosis_1, n_samples = low_risk_diagnosis.shape[0])
rare_diagnosis_upsample_2 = resample(rare_diagnosis_2, n_samples = low_risk_diagnosis.shape[0])

heart_disease = pd.concat((rare_diagnosis_upsample_1, rare_diagnosis_upsample_2, low_risk_diagnosis)).reset_index(drop = True)

heart_disease

,cholesterol,type_chestpain,max_heart_rate,resting_bp,diagnosis
0,212,type4,132,145,high-risk heart disease
1,206,type4,132,130,high-risk heart disease
2,318,type4,140,114,high-risk heart disease
3,289,type4,145,160,high-risk heart disease
4,230,type3,165,112,high-risk heart disease
...,...,...,...,...,...
652,157,type2,182,120,low-risk heart disease
653,241,type4,123,140,low-risk heart disease
654,264,type1,132,110,low-risk heart disease
655,236,type2,174,130,low-risk heart disease


In [6]:
# split data into training and test sets
heart_disease_train, heart_disease_test = train_test_split(heart_disease, train_size = 0.75, random_state = 0)

In [7]:
# Summary of the categorical variables
heart_disease_categorical = heart_disease_train.drop(columns = ["cholesterol","max_heart_rate","resting_bp"])
heart_disease_categorical.describe()

,type_chestpain,diagnosis
count,492,492
unique,4,3
top,type4,low-risk heart disease
freq,335,168


In [8]:
# Summary of the continuous variables
heart_disease_continuous = heart_disease_train.drop(columns = ["type_chestpain","diagnosis"])
heart_disease_continuous.describe()

,cholesterol,max_heart_rate,resting_bp
count,492.000000,492.000000,492.000000
mean,250.619919,143.544715,135.910569
std,56.695060,21.078653,18.868182
min,131.000000,71.000000,94.000000
25%,211.000000,128.000000,120.000000
50%,243.000000,143.000000,134.000000
75%,289.000000,161.000000,150.000000
max,564.000000,202.000000,200.000000


In [9]:
# blood pressure
bp_hist = alt.Chart(heart_disease_train).mark_bar().encode(
    x=alt.X("resting_bp:Q", bin = True).title("Blood Pressure"),
    y=alt.Y("count()").stack(False),
    color="diagnosis:N"
).properties(
    title = "Distribution of Blood Pressure"
)

bp_hist

alt.Chart(...)

In [10]:
# cholesterol
chol_hist = alt.Chart(heart_disease_train).mark_bar().encode(
    x=alt.X("cholesterol:Q", bin = True).title("Cholesterol"),
    y=alt.Y("count()").stack(False),
    color = "diagnosis:N"
).properties(
    title = "Distribution of Cholesterol"
)

chol_hist

alt.Chart(...)

In [11]:
# chest pain type
cp_hist = alt.Chart(heart_disease_train).mark_bar().encode(
    x=alt.X("type_chestpain").title("Chest Pain Type"),
    y=alt.Y("count()").stack(False),
    color = "diagnosis:N"
).properties(
    width=300,
    height=300,
    title = "Distribution of Chest Pain Type"
)

cp_hist

alt.Chart(...)

In [12]:
# heart rate
hr_hist = alt.Chart(heart_disease_train).mark_bar().encode(
    x=alt.X("max_heart_rate:Q", bin = True).title("Heart Rate"),
    y=alt.Y("count()").stack(False),
    color = "diagnosis:N"
).properties(
    title = "Distribution of Heart Rate"
)

hr_hist

alt.Chart(...)

In [13]:
preprocessor=make_column_transformer(
    (StandardScaler(),['cholesterol','max_heart_rate','resting_bp']),
    remainder='passthrough',
    verbose_feature_names_out=False
)
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('standardscaler', StandardScaler(),
                                 ['cholesterol', 'max_heart_rate',
                                  'resting_bp'])],
                  verbose_feature_names_out=False)

In [14]:
# create a pipeline
heart_disease_pipe = make_pipeline(preprocessor, KNeighborsClassifier())
heart_disease_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['cholesterol',
                                                   'max_heart_rate',
                                                   'resting_bp'])],
                                   verbose_feature_names_out=False)),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [15]:
np.random.seed(1234)
parameter_grid = {
    "kneighborsclassifier__n_neighbors" : range(1, 31)
}

grid_search = GridSearchCV(
    estimator = heart_disease_pipe,
    param_grid = parameter_grid,
    cv = 5,
)


grid_search

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('standardscaler',
                                                                         StandardScaler(),
                                                                         ['cholesterol',
                                                                          'max_heart_rate',
                                                                          'resting_bp'])],
                                                          verbose_feature_names_out=False)),
                                       ('kneighborsclassifier',
                                        KNeighborsClassifier())]),
             param_grid={'kneighborsclassifier__n_neighbors': range(1, 31)})

In [16]:
X_heart_train=heart_disease_train[['cholesterol','max_heart_rate','resting_bp']]
y_heart_train=heart_disease_train['diagnosis']

model_grid=grid_search.fit(X_heart_train,y_heart_train)
grid_results=pd.DataFrame(grid_search.cv_results_)
grid_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kneighborsclassifier__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007334,0.000909,0.010141,0.001598,1,{'kneighborsclassifier__n_neighbors': 1},0.868687,0.868687,0.846939,0.785714,0.857143,0.845434,0.030943,1
1,0.008843,0.001838,0.012545,0.005679,2,{'kneighborsclassifier__n_neighbors': 2},0.777778,0.858586,0.806122,0.724490,0.775510,0.788497,0.043826,2
2,0.007401,0.000565,0.009990,0.002323,3,{'kneighborsclassifier__n_neighbors': 3},0.787879,0.828283,0.795918,0.714286,0.704082,0.766089,0.048500,3
3,0.008447,0.000901,0.019641,0.016322,4,{'kneighborsclassifier__n_neighbors': 4},0.787879,0.787879,0.734694,0.673469,0.704082,0.737600,0.045388,4
4,0.008158,0.001609,0.009604,0.000897,5,{'kneighborsclassifier__n_neighbors': 5},0.747475,0.777778,0.755102,0.653061,0.744898,0.735663,0.042894,5
5,0.008970,0.002479,0.011178,0.002078,6,{'kneighborsclassifier__n_neighbors': 6},0.727273,0.767677,0.724490,0.602041,0.673469,0.698990,0.056953,6
6,0.007466,0.000459,0.009878,0.002756,7,{'kneighborsclassifier__n_neighbors': 7},0.666667,0.737374,0.734694,0.663265,0.663265,0.693053,0.035126,7
7,0.008148,0.001150,0.010933,0.001608,8,{'kneighborsclassifier__n_neighbors': 8},0.696970,0.656566,0.693878,0.663265,0.653061,0.672748,0.018828,8
8,0.007624,0.000748,0.009813,0.001040,9,{'kneighborsclassifier__n_neighbors': 9},0.676768,0.646465,0.683673,0.663265,0.673469,0.668728,0.012930,9
9,0.007743,0.001606,0.010119,0.001562,10,{'kneighborsclassifier__n_neighbors': 10},0.707071,0.666667,0.653061,0.632653,0.683673,0.668625,0.025474,10


In [17]:
cross_val_plot = alt.Chart(grid_results).mark_line(point=True).encode(
    x=alt.X("param_kneighborsclassifier__n_neighbors").title("Values for K").scale(zero=True),
    y=alt.Y("mean_test_score").title("Accuracy of model").scale(zero=False)
)

cross_val_plot

alt.Chart(...)

In [18]:
# we can see that the best value for k is 1
knn=KNeighborsClassifier(n_neighbors=1)
heart_fit=knn.fit(X_heart_train,y_heart_train)
heart_predictions = heart_disease_test.assign(predicted = heart_fit.predict(heart_disease_test[['cholesterol', 'max_heart_rate', 'resting_bp']]))
heart_predictions.head(50)

,cholesterol,type_chestpain,max_heart_rate,resting_bp,diagnosis,predicted
538,245,type3,166,125,low-risk heart disease,low-risk heart disease
493,197,type4,177,110,low-risk heart disease,low-risk heart disease
14,225,type4,114,150,high-risk heart disease,high-risk heart disease
247,260,type4,140,120,moderate-risk heart disease,moderate-risk heart disease
85,230,type3,165,112,high-risk heart disease,high-risk heart disease
127,166,type4,125,138,high-risk heart disease,high-risk heart disease
301,188,type3,139,120,moderate-risk heart disease,moderate-risk heart disease
532,227,type3,154,94,low-risk heart disease,moderate-risk heart disease
331,294,type4,106,138,moderate-risk heart disease,moderate-risk heart disease
484,177,type3,160,142,low-risk heart disease,moderate-risk heart disease


In [19]:
# test model's accuracy 
heart_disease_correct = heart_predictions[
    heart_predictions['diagnosis'] == heart_predictions['predicted']
] 
heart_disease_acc = heart_disease_correct.shape[0] / heart_predictions.shape[0]
heart_disease_acc

0.9030303030303031

In [20]:
# confusion matrix
confusion_matrix = pd.crosstab(
    heart_predictions['diagnosis'],
    heart_predictions['predicted'],
    rownames=['Actual'],
    colnames=['Predicted']
)

confusion_matrix

Predicted,high-risk heart disease,low-risk heart disease,moderate-risk heart disease
Actual,,,
high-risk heart disease,54,0,0
low-risk heart disease,2,39,10
moderate-risk heart disease,0,4,56


In [86]:
predictions_hist = alt.Chart(heart_predictions).mark_bar().encode(
    x = alt.X('predicted').title('Predicted Diagnosis'),
    y = alt.Y('count()').title('Count of Predicted Diagnosis'),
    column = alt.Column('diagnosis').title('Diagnosis'),
    color = alt.Color('predicted').title('Predicted Diagnosis'),
).properties(
    height = 350,
    width = 350,
).configure_axisX(labelAngle = -40, titleFontSize = 12).configure_axis(titleFontSize = 12, labelFontSize = 12, labelAlign = 'center', labelPadding = 40)

predictions_hist

alt.Chart(...)